In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.model_selection import GridSearchCV

from sklearn.metrics import roc_auc_score ## To compute the auc score
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier

from mlxtend.classifier import EnsembleVoteClassifier
from sklearn.naive_bayes import GaussianNB 
from sklearn.ensemble import VotingClassifier

from sklearn.ensemble import GradientBoostingClassifier
from sklearn import model_selection

from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer

## Add cross validation

In [38]:
## FOR NORMALIZATION
data = pd.read_csv('train_2008.csv')
y_train = data.pop('target').values
X_train = preprocessing.scale(np.array(data))

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/utils/validation.py:590: DataConversionWarning: Data with input dtype int64 was converted to float64 by the scale function.
  warnings.warn(msg, DataConversionWarning)


In [9]:
from sklearn import model_selection

def cross_val(clf, X_train, y_train):
    
    result = model_selection.cross_validate(clf, X_train, y_train, scoring='roc_auc', cv=10, return_train_score=True)
    return result

In [21]:
def process_for_submission(y_pred, title):
    lst = y_pred
    print(len(lst))
    
    indexes = [i for i in range(len(lst))]
    result = []
    result.append(indexes)
    result.append(lst)
    result = np.array(result)
    result = np.transpose(result)

    
    
    #df = df.astype(int)
    df = pd.DataFrame(result, columns=['id', 'target'])
    df["id"] = df["id"].astype("int")
    df["target"] = df["target"].astype("float")
    df = df.to_csv (title + 'submission.csv', index = None, header=True)

    
    return df
    

In [93]:
process_for_submission(gbm.predict_proba(X_real)[:, 1])

16000
16000


In [2]:
import h2o
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
  Java Version: java version "1.8.0_161"; Java(TM) SE Runtime Environment (build 1.8.0_161-b12); Java HotSpot(TM) 64-Bit Server VM (build 25.161-b12, mixed mode)
  Starting server from /Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /var/folders/cv/xqmn6z1x6l39r_575wkj9b780000gn/T/tmp9_u_uf97
  JVM stdout: /var/folders/cv/xqmn6z1x6l39r_575wkj9b780000gn/T/tmp9_u_uf97/h2o_pavanchitta_started_from_python.out
  JVM stderr: /var/folders/cv/xqmn6z1x6l39r_575wkj9b780000gn/T/tmp9_u_uf97/h2o_pavanchitta_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321... successful.


H2O cluster uptime:,01 secs
H2O cluster timezone:,America/Los_Angeles
H2O data parsing timezone:,UTC
H2O cluster version:,3.22.1.3
H2O cluster version age:,16 days
H2O cluster name:,H2O_from_python_pavanchitta_r1zp2a
H2O cluster total nodes:,1
H2O cluster free memory:,3.556 Gb
H2O cluster total cores:,8
H2O cluster allowed cores:,8
H2O cluster status:,"accepting new members, healthy"


In [24]:
def drop_columns(data):
    drop_columns = []
    for column in data.columns:
        if (data[column] == -1).sum() / data.shape[0] > 0.95:
            drop_columns.append(column)
    return data.drop(columns=drop_columns), drop_columns

# To be performed after drop columns is called!!!
def one_hot_encoding(data, categoricals):
    categorical_idx = []
    count = 0
    for column in categoricals:
        if column in data and data[column].nunique() < 10  and (data[column] < 0).sum() == 0:
            count += 1
            #print(data.count_vals)
            categorical_idx.append(data.columns.get_loc(column))
    print(count)
    enc = OneHotEncoder(categorical_features=categorical_idx)
    return enc.fit_transform(data).toarray(), categorical_idx
    

In [25]:
data = pd.read_csv('train_2008.csv')

sm_results = pd.read_csv('smart_mapping_results.csv')
idx = list(sm_results['Feature']).index('HUFINAL')
drop_cols_sm = list(sm_results['Feature'])[idx:]
data.drop(columns=drop_cols_sm, inplace=True)

data, drop_columns = drop_columns(data)
data = data.drop(columns=['id'])
y_train = data.pop('target').values

categoricals =['HURESPLI', 'HUFINAL', 'HUSPNISH', 'HETENURE', 'HEHOUSUT', \
               'HETELHHD', 'HETELAVL', 'HEPHONEO', 'HUTYPEA', 'HUTYPB', 'HUTYPC', \
               'HRINTSTA', 'HRHTYPE', 'HUINTTYP', 'HRLONGLK', 'HUBUS', 'GEREG', \
               'GESTCEN', 'GESTFIPS', 'GTCBSA', 'GTCO', 'GTCBSAST', 'GTMETSTA', \
               'PUPELIG', 'PERRP', 'PRTFAGE', 'PEMARITL', 'PESEX', 'PEAFEVER', \
               'PEAFNOW', 'PEEDUCA', 'PTDTRACE', 'PRDTHSP', 'PUCHINHH', 'PRFAMNUM', \
               'PRFAMREL', 'PRFAMTYP', 'PEHSPNON', 'PRMARSTA', 'PRPERTYP', \
               'PENATVTY', 'PEMNTVTY', 'PEFNTVTY', 'PRCITSHP', 'PUSLFPRX', \
               'PEMLR', 'PUWK', 'PUBUS1', 'PUBUS2OT', 'PUBUSCK1', 'PUBUSCK2', \
               'PUBUSCK3', 'PUBUSCK4', 'PURETOT', 'PUDIS', 'PERET1', 'PUDIS1', \
               'PUDIS2', 'PUABSOT', 'PULAY', 'PEABSRSN', 'PEABSPDO', 'PEMJOT', \
               'PEHRFTPT', 'PEHRUSLT', 'PEHRWANT', 'PEHRRSN1', 'PEHRRSN2', 'PEHRRSN3', \
               'PUHROFF1', 'PUHROT1', 'PEHRAVL', 'PULAYDT', 'PULAY6M', 'PELAYAVL', \
               'PULAYAVR', 'PELAYLK', 'PELAYFTO', 'PULK', 'PELKM1', 'PULKM2', \
               'PULKM3', 'PULKM4', 'PULKM5', 'PULKM6', 'PULKDK1', 'PULKDK2', \
               'PULKPS1', 'PULKPS2', 'PULKPS3', 'PELKAVL', 'PULKAVR', 'PELKLL1O', \
               'PELKLL2O', 'PELKLWO', 'PELKFTO', 'PEDWWNTO', 'PEDWRSN', 'PEDWLKO', \
               'PEDWWK', 'PEDW4WK', 'PEDWLKWK', 'PEDWAVL', 'PEDWAVR', 'PUDWCK1', 'PUJHDP1O', 'PEJHRSN', 'PEJHWANT', 'PRABSREA', 'PRCIVLF', 'PRDISC', 'PREMPHRS', 'PREMPNOT', 'PREXPLF', 'PRFTLF', 'PRJOBSEA', 'PRPTREA', 'PRUNTYPE', 'PRWKSCH', 'PRWKSTAT', 'PRWNTJOB', 'PUJHCK3', 'PUJHCK3', 'PUJHCK4', 'PUJHCK5', 'PUIODP1', 'PUIODP2', 'PUIODP3', 'PEIO1COW', 'PUIO1MFG', 'PEIO2COW', 'PUIO2MFG', 'PUIOCK1', 'PUIOCK2', 'PRIOELG', 'PRAGNA', 'PRCOW1', 'PRCOW2', 'PRCOWPG', 'PRDTCOW1', 'PRDTCOW2', 'PRDTIND1', 'PRDTIND2', 'PRDTOCC1', 'PRDTOCC2', 'PREMP', 'PRMJIND1', 'PRMJIND2', 'PRMJOCC1', 'PRMJOCC2', 'PRMJOCGR', 'PRNAGPWS', 'PRNAGWS', 'PRSJMJ', 'PRERELG', 'PEERNUOT', 'PEERNPER', 'PEERNRT', 'PEERNHRY', 'PEERNLAB', 'PEERNCOV', 'PENLFJH', 'PENLFRET', 'PENLFACT', 'PESCHENR', 'PESCHFT', 'PESCHLVL', 'PRNLFSCH', 'PRWERNAL', 'PRHERNAL', 'HXTENURE', 'HXHOUSUT', 'HXHOUSUT', 'HXTELAVL', 'HXPHONEO', 'PXINUSYR', 'PXRRP', 'PXPARENT', 'PXAGE', 'PXMARITL', 'PXSPOUSE', 'PXSEX', 'PXAFWHN1', 'PXAFNOW', 'PXEDUCA', 'PXRACE1', 'PXNATVTY', 'PXMNTVTY', 'PXFNTVTY', 'PXHSPNON', 'PXMLR', 'PXRET1', 'PXABSRSN', 'PXABSPDO', 'PXMJOT', 'PXMJNUM', 'PXMJNUM', 'PXHRUSL2', 'PXHRFTPT', 'PXHRUSLT', 'PXHRWANT', 'PXHRRSN1', 'PXHRRSN2', 'PXHRACT1', 'PXHRACT2', 'PXHRACTT', 'PXHRRSN3', 'PXHRAVL', 'PXLAYAVL', 'PXLAYLK', 'PXLAYDUR', 'PXLAYFTO', 'PXLKM1', 'PXLKAVL', 'PXLKLL1O', 'PXLKLL2O', 'PXLKLWO', 'PXLKDUR', 'PXLKFTO', 'PXDWWNTO', 'PXDWRSN', 'PXDWLKO', 'PXDWWK', 'PXDW4WK', 'PXDWLKWK', 'PXDWAVL', 'PXDWAVR', 'PXJHWKO', 'PXJHRSN', 'PXJHWANT', 'PXIO1COW', 'PXIO1ICD', 'PXIO1OCD', 'PXIO2COW', 'PXIO2ICD', 'PXIO2OCD', 'PXERNUOT', 'PXERNPER', 'PXERNH1O', 'PXERNWKP', 'PXERNRT', 'PXERNHRY', 'PXERNH2', 'PXERNLAB', 'PXERNCOV', 'PXNLFJH', 'PXNLFRET', 'PXNLFACT', 'PXSCHENR', 'PXSCHFT', 'PXSCHFT', 'PEDIPGED', 'PEHGCOMP', 'PEHGCOMP', 'PEGR6COR', 'PEMS123', 'PXDIPGED', 'PXHGCOMP', 'PXCYC', 'PXGRPROF', 'PXGR6COR', 'PXMS123', 'PEIO1ICD', 'PEIO2ICD', 'PEIO2OCD', 'PRIMIND1', 'PRIMIND2', 'PEAFWHN1', 'PEAFWHN2', 'PEAFWHN3', 'PEAFWHN4', 'PXAFEVER', 'PELNDAD', 'PEDADTYP', 'PEMOMTYP', 'PXLNDAD', 'PXLNMOM', 'PXDADTYP', 'PXMOMTYP', 'PXCOHAB', 'PEDISEAR', 'PEDISEYE', 'PEDISREM', 'PEDISPHY', 'PEDISDRS', 'PEDISOUT', 'PRDISFLG', 'PXDISEAR', 'PXDISEYE', 'PXDISREM', 'PXDISPHY', 'PXDISDRS', 'PXDISOUT', 'PES1', 'PES2', 'PES3', 'PES4', 'PES5', 'PES6', 'PES7', 'PUSCK4']

In [26]:
#data_arr, categorical_idx = one_hot_encoding(data, categoricals)
#data_train_h2o=h2o.H2OFrame(data_arr)
#data_labels_h2o = h2o.H2OFrame(y_train)

real_data = pd.read_csv('test_2008.csv')
real_data.drop(columns=drop_cols_sm, inplace=True)
real_data = real_data.drop(columns=drop_columns)
real_data = real_data.drop(columns=['id'])


# PERFORM CORRECT IMPUTATION. SHOULD FIT BASED ON THE TRAIN DATA AND THEN IMPUTE
train_cols = data.columns
imp_mean = SimpleImputer(missing_values=-1, strategy='mean')
data = pd.DataFrame(imp_mean.fit_transform(np.array(data)))
data = data.round().astype(int)
data.columns = train_cols

test_cols = real_data.columns
real_data = pd.DataFrame(imp_mean.transform(np.array(real_data)))
real_data.columns = test_cols
real_data = real_data.round().astype(int)

# Concatenate the two in order to assign as categorical
combined_data = pd.concat([data, real_data])
print(combined_data.head())
combined_data_h2o=h2o.H2OFrame(combined_data)

# Assign categorical features
for col_name in categoricals:
    if col_name in combined_data_h2o.names:
        combined_data_h2o[col_name] = combined_data_h2o[col_name].asfactor()
        


   HRMONTH  HRYEAR4  HURESPLI  HETENURE  HEHOUSUT  HETELHHD  HETELAVL  \
0       11     2008         1         1         1         1         1   
1       11     2008         1         2         1         1         1   
2       11     2008         1         1         1         1         1   
3       11     2008         2         1         1         1         1   
4       11     2008         2         1         1         1         1   

   HEPHONEO  HUFAMINC   HWHHWGT    ...     PXMOMTYP  PEDISEAR  PEDISREM  \
0         1        16  39202259    ...            1         2         2   
1         1         6  41152903    ...            1         2         2   
2         1        15  30422918    ...            1         2         2   
3         1        13  15096027    ...            1         2         2   
4         1        -3  28225917    ...            0         2         2   

   PEDISPHY  PEDISOUT  PRDISFLG  PXDISEAR  PXDISEYE  PXDISDRS  PXDISOUT  
0         2         2         2     

## RUN THE FOLLOWING COMMAND ONLY FOR MY IMPLEMENTATION OF ONE HOT ENCODING

In [290]:
combined_data, categorical_idx = one_hot_encoding(combined_data, categoricals)

120


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/preprocessing/_encoders.py:363: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/preprocessing/_encoders.py:385: DeprecationWarning: The 'categorical_features' keyword is deprecated in version 0.20 and will be removed in 0.22. You can use the ColumnTransformer instead.
  "use the ColumnTransformer instead.", DeprecationWarning)


In [27]:
"""train_data = combined_data[:64667, :]
train_data = np.column_stack((train_data, y_train[:]))
data_train_h2o = h2o.H2OFrame(train_data)
data_train_h2o[-1]=data_train_h2o[-1].asfactor()
real_data = combined_data[64667:]
data_test_h2o=h2o.H2OFrame(real_data)"""
data_train_h2o = combined_data_h2o[:64667, :]
y_train_h2o = h2o.H2OFrame(y_train)
#train_data = np.column_stack((np.array(data_train_data_h2o), y_train[:]))
data_train_h2o = data_train_h2o.cbind(y_train_h2o)
#data_train_h2o = h2o.H2OFrame(train_data)
data_train_h2o[-1]=data_train_h2o[-1].asfactor()
data_test_h2o = combined_data_h2o[64667:, :]
#data_test_h2o=h2o.H2OFrame(real_data)

Parse progress: |█████████████████████████████████████████████████████████| 100%


## Only for grid search 


In [52]:
from h2o.grid.grid_search import H2OGridSearch
from h2o.estimators.gbm import H2OGradientBoostingEstimator
grid_search_gbm = H2OGradientBoostingEstimator(
    
  ## early stopping once the validation AUC doesn't improve by at least 0.01% for 5 consecutive scoring events
  stopping_rounds = 5, stopping_tolerance = 1e-4, stopping_metric = "AUC", 

  ## sample 80% of rows per tree
  sample_rate = 1.0,                                                       

  ## sample 80% of columns per split
  col_sample_rate = 0.8,                                                   

  ## fix a random number generator seed for reproducibility
  seed = 1234,                                                             

  ## score every 10 trees to make early stopping reproducible (it depends on the scoring interval)
  score_tree_interval = 0
) 

ss = data_train_h2o.split_frame(seed = 1)
train = ss[0]
val = ss[1]

hyper_params2 = {
    'learn_rate':[0.01, 0.02],
    'max_depth':[2,4,6],
    'ntrees':[500, 1000, 5000]}

In [53]:
grid = H2OGridSearch(grid_search_gbm, hyper_params2,
                         grid_id='depth_grid',
                         search_criteria={'strategy': "Cartesian"})
#Train grid search
grid.train(x=data_train_h2o.names[:255], 
           y=data_train_h2o.names[255],
           training_frame=train,
           validation_frame=val)

gbm Grid Build progress: |████████████████████████████████████████████████| 100%


In [23]:
grid_sorted = grid.get_grid(sort_by='auc',decreasing=True)

In [38]:
print(grid_sorted.models[0].params['learn_rate'])

{'default': 0.1, 'actual': 0.02}


In [39]:
best_gbm = grid_sorted.models[0]
best_gbm.train(x=data_train_h2o.names[:255],\
            y=data_train_h2o.names[255], training_frame=data_train_h2o, model_id="GBM_Titanic")

gbm Model Build progress: |███████████████████████████████████████████████| 100%


In [42]:
f=best_gbm.predict(test_data=data_test_h2o)

gbm prediction progress: |████████████████████████████████████████████████| 100%


In [43]:
f = f.as_data_frame()

In [47]:
preds = np.array(f['p1'])
process_for_submission(preds, 'h2o_gbm4')

16000


## Train an h2o gbm

In [28]:
from h2o.estimators.gbm import H2OGradientBoostingEstimator


In [29]:
# try a range of values for `nbins`

ss = data_train_h2o.split_frame(seed = 1)
train = ss[0]
val = ss[1]

nbins_cats = [128]#[512, 256]#[1024, 2048]
label = ['128']#["512", "256"]#["1024", "2048"]
for key, num in enumerate(nbins_cats):
    # initialize the GBM estimator and set a seed for reproducibility
    eeg_gbm = H2OGradientBoostingEstimator(
        
  nbins_cats=num,
  ntrees = 14000,                                                            
  learn_rate = 0.01,                                                         
  stopping_rounds = 25, stopping_tolerance = 1e-4, stopping_metric = "AUC", 
  sample_rate = 0.9,                                                       
  col_sample_rate = 0.8,                                                   
  ## fix a random number generator seed for reproducibility
  seed = 1234,                                                             
  score_tree_interval = 0, max_depth=2)
    
    eeg_gbm.train(x=data_train_h2o.names[:-1],y=data_train_h2o.names[-1], training_frame=train, validation_frame=val, model_id="GBM_Titanic")
    # print the value used and AUC score for train and validation sets
    print(label[key], 'training score', eeg_gbm.auc(train = True))
    print(label[key], 'validation score', eeg_gbm.auc(valid = True))

gbm Model Build progress: |███████████████████████████████████████████████| 100%
128 training score 0.8336599760643708
128 validation score 0.7881682091488975


In [13]:
model = H2OGradientBoostingEstimator(## more trees is better if the learning rate is small enough 
  
  
  nbins_cats=256  ,
  ## here, use "more than enough" trees - we have early stopping
  ntrees = 14000,                                                            

  ## smaller learning rate is better (this is a good value for most datasets, but see below for annealing)
  learn_rate = 0.01,                                                         

  ## early stopping once the validation AUC doesn't improve by at least 0.01% for 5 consecutive scoring events
  stopping_rounds = 25, stopping_tolerance = 1e-4, stopping_metric = "AUC", 

  ## sample 80% of rows per tree
  sample_rate = 0.8,                                                       

  ## sample 80% of columns per split
  col_sample_rate = 0.8,                                                   

  ## fix a random number generator seed for reproducibility
  seed = 1234,                                                             

  ## score every 10 trees to make early stopping reproducible (it depends on the scoring interval)
  score_tree_interval = 0, max_depth=2)

In [14]:
model.train(x=data_train_h2o.names[:-1],y=data_train_h2o.names[-1], training_frame=data_train_h2o, model_id="GBM_Titanic")

gbm Model Build progress: |███████████████████████████████████████████████| 100%


In [15]:
model

Model Details
H2OGradientBoostingEstimator :  Gradient Boosting Machine
Model Key:  GBM_Titanic


ModelMetricsBinomial: gbm
** Reported on train data. **

MSE: 0.1267312986504977
RMSE: 0.3559933969197992
LogLoss: 0.40033205701290314
Mean Per-Class Error: 0.22755018772675506
AUC: 0.854706067656277
pr_auc: 0.7000880908118873
Gini: 0.709412135312554
Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.3053092567221996: 


,0,1,Error,Rate
0,39457.0,8695.0,0.1806,(8695.0/48152.0)
1,4596.0,11919.0,0.2783,(4596.0/16515.0)
Total,44053.0,20614.0,0.2055,(13291.0/64667.0)


Maximum Metrics: Maximum metrics at their respective thresholds



metric,threshold,value,idx
max f1,0.3053093,0.6420318,221.0
max f2,0.1807476,0.7348308,284.0
max f0point5,0.4808243,0.6659067,148.0
max accuracy,0.4671452,0.8236968,153.0
max precision,0.9893264,1.0,0.0
max recall,0.0181846,1.0,393.0
max specificity,0.9893264,1.0,0.0
max absolute_mcc,0.3712259,0.5132610,192.0
max min_per_class_accuracy,0.2660039,0.7705117,239.0
max mean_per_class_accuracy,0.2867498,0.7724498,229.0


Gains/Lift Table: Avg response rate: 25.54 %, avg score: 25.54 %



...


Scoring History: 


,timestamp,duration,number_of_trees,training_rmse,training_logloss,training_auc,training_pr_auc,training_lift,training_classification_error
,2019-02-10 20:41:56,0.001 sec,0.0,0.4360776,0.5681745,0.5,0.0,1.0,0.7446147
,2019-02-10 20:41:56,0.116 sec,1.0,0.4356489,0.5671931,0.6924654,0.3020837,2.0683960,0.3456477
,2019-02-10 20:41:56,0.171 sec,2.0,0.4352350,0.5662482,0.6924654,0.3020837,2.0683960,0.3456477
,2019-02-10 20:41:56,0.232 sec,3.0,0.4348333,0.5653345,0.7002938,0.3038760,2.0683960,0.3754001
,2019-02-10 20:41:56,0.318 sec,4.0,0.4344060,0.5643636,0.7181679,0.3671334,2.2882948,0.4040392
---,---,---,---,---,---,---,---,---,---
,2019-02-10 20:55:03,13 min 7.240 sec,13754.0,0.3563423,0.4009923,0.8540853,0.6987668,3.7885602,0.2055608
,2019-02-10 20:55:07,13 min 11.253 sec,13826.0,0.3562361,0.4007927,0.8542588,0.6992305,3.7885602,0.2013392
,2019-02-10 20:55:11,13 min 15.270 sec,13894.0,0.3561369,0.4006119,0.8544329,0.6995602,3.7885602,0.2037206
,2019-02-10 20:55:15,13 min 19.271 sec,13965.0,0.3560363,0.4004131,0.8546194,0.6998522,3.7825082,0.2054989



See the whole table with table.as_data_frame()
Variable Importances: 


variable,relative_importance,scaled_importance,percentage
PEEDUCA,49226.6054688,1.0,0.2456579
GTCO,20755.3691406,0.4216291,0.1035765
PEAGE,16600.7304688,0.3372309,0.0828434
GESTCEN,11954.6201172,0.2428488,0.0596577
GESTFIPS,11641.0878906,0.2364796,0.0580931
---,---,---,---
PEGRPROF,0.0,0.0,0.0
PEMS123,0.0,0.0,0.0
PELNMOM,0.0,0.0,0.0
PEDISEAR,0.0,0.0,0.0



See the whole table with table.as_data_frame()


In [16]:
f=model.predict(test_data=data_test_h2o)

gbm prediction progress: |████████████████████████████████████████████████| 100%


In [17]:
f = f.as_data_frame()

In [18]:
print(f)

       predict        p0        p1
0            0  0.830422  0.169578
1            0  0.880954  0.119046
2            0  0.866003  0.133997
3            1  0.524780  0.475220
4            0  0.779094  0.220906
5            0  0.947222  0.052778
6            1  0.410961  0.589039
7            1  0.241494  0.758506
8            0  0.979206  0.020794
9            0  0.877659  0.122341
10           0  0.968406  0.031594
11           1  0.371939  0.628061
12           0  0.778136  0.221864
13           0  0.731845  0.268155
14           0  0.712369  0.287631
15           0  0.955035  0.044965
16           1  0.640911  0.359089
17           0  0.945110  0.054890
18           0  0.789275  0.210725
19           1  0.648079  0.351921
20           0  0.824178  0.175822
21           0  0.885384  0.114616
22           0  0.937587  0.062413
23           0  0.955140  0.044860
24           1  0.366570  0.633430
25           0  0.974835  0.025165
26           0  0.980029  0.019971
27           1  0.56

In [19]:
preds = np.array(f['p1'])

In [22]:
process_for_submission(preds, 'h2o_gbm16')

16000


In [23]:
model_path = h2o.save_model(model=model, path="./mymodel", force=True)

In [203]:
print(model_path)

/Users/pavanchitta/CS155-Project1/mymodel/GBM_Titanic


In [204]:
h2o_model1 = pd.read_csv('h2o_gbm3submission.csv')
h2o_model2 = pd.read_csv('h2o_gbm6submission.csv')

In [133]:
comb_h2o = pd.concat([h2o_model1, h2o_model2]).groupby(level=0).mean()

In [134]:
comb_h2o.to_csv('comb_h2o_submission.csv', index=None)

In [267]:
saved_model2 = h2o.load_model('/Users/pavanchitta/CS155-Project1/mymodel/h2o_gbm_15000_to_submit')

In [269]:
f=saved_model2.predict(test_data=data_test_h2o)

gbm prediction progress: |████████████████████████████████████████████████| 100%
